# Analyse du data drift

In [1]:
import pandas as pd

import numpy as np
from evidently.report import Report
from evidently.metrics import DatasetDriftMetric
from evidently.metric_preset import TargetDriftPreset
from evidently.metrics import DataDriftTable
import warnings
warnings.filterwarnings("ignore")

C:\Users\mr_ar\anaconda3\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\mr_ar\anaconda3\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\mr_ar\anaconda3\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning: The 'nopython' keywo

In [2]:
#import data
train_df=pd.read_csv('df_train_api.csv')
test_df=pd.read_csv('df_test_api.csv')

In [3]:
train_df=train_df.drop(columns=['Unnamed: 0','index','TARGET',"SK_ID_CURR"],axis=1)
test_df=test_df.drop(columns=["SK_ID_CURR"],axis=1)

In [4]:
from evidently.pipeline.column_mapping import ColumnMapping
column_mapping = ColumnMapping()

In [5]:
import joblib
model = joblib.load("model.joblib")
# we do not apply SMOTE on test data
model.steps.pop(0)
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('Logreg',
                 LogisticRegression(C=100, class_weight='balanced',
                                    max_iter=10000))])

In [6]:
test_df['prediction'] = model.predict(test_df)
train_df['prediction'] = model.predict(train_df)

In [7]:
features=[#'CC_AMT_BALANCE_VAR',
          # 'CC_AMT_TOTAL_RECEIVABLE_VAR',
          #'CC_AMT_RECIVABLE_VAR',
          #'CC_AMT_CREDIT_LIMIT_ACTUAL_VAR',
          'prediction'
    ]

In [8]:
column_mapping = ColumnMapping()
column_mapping.features = features

In [9]:
report = Report(metrics=[
    TargetDriftPreset()
])

report.run(reference_data=train_df, current_data=test_df,#column_mapping=column_mapping
          )
report.save_html('evidently_metrics_report.html')